In [1]:
%reload_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import torch
import torchvision
import torchprof

# Define a model and input

In [ ]:
model = torchvision.models.alexnet(pretrained=False).cuda()
x = torch.rand([1, 3, 224, 224]).cuda()

# Run a profile

In [5]:
with torchprof.profile(model, use_cuda=True) as prof:
    _ = model(x)


Adding traces:   0%|          | 0/24 [00:00<?, ?it/s]

Adding traces: 100%|██████████| 24/24 [00:00<00:00, 96513.23it/s]


Removing traces:   0%|          | 0/24 [00:00<?, ?it/s]

Removing traces: 100%|██████████| 24/24 [00:00<00:00, 200125.84it/s]

# Displaying profile

In [6]:
prof.display()

Make Events:   0%|          | 0/252 [00:00<?, ?it/s]

Make Events: 100%|██████████| 252/252 [00:00<00:00, 62017.52it/s]


Populate ancestors:   0%|          | 0/252 [00:00<?, ?it/s]

Populate ancestors: 100%|██████████| 252/252 [00:00<00:00, 302162.55it/s]


Group:   0%|          | 0/252 [00:00<?, ?it/s]

...


Aggregate:   0%|          | 0/177 [00:00<?, ?it/s]

Aggregate: 100%|██████████| 177/177 [00:00<00:00, 81082.55it/s]


Make rows: 0it [00:00, ?it/s]

Make rows: 26it [00:00, 1101.56it/s]



CPU=14.7ms, CUDA=14.6ms
+------------------+--------------+---------------+--------------+---------------+-----+
| Node             |     Self CPU |           CPU |    Self CUDA |          CUDA |   # |
|------------------+--------------+---------------+--------------+---------------+-----|
| <torchprof>      | 392.2µs (3%) | 14.6ms (100%) | 393.4µs (3%) | 14.6ms (100%) |   1 |
|  └──AlexNet      | 236.6µs (2%) |  14.2ms (97%) | 238.5µs (2%) |  14.2ms (97%) |   1 |
|    ├──classifier | 461.0µs (3%) |   8.0ms (55%) | 462.0µs (3%) |   8.0ms (55%) |   1 |
|    │ ├──1        |  4.4ms (30%) |   4.5ms (31%) |              |   4.5ms (31%) |   1 |
|    │ ├──4        |  1.9ms (13%) |   2.0ms (14%) |              |   2.0ms (14%) |   1 |
|    │ ├──6        | 482.9µs (3%) |  564.8µs (4%) |              |  565.1µs (4%) |   1 |
|    ├──features   | 815.6µs (6%) |   5.8ms (39%) | 823.2µs (6%) |   5.8ms (39%) |   1 |
|    │ ├──8        | 679.6µs (5%) |  850.2µs (6%) |              |  849.9µs (6%) |  

In [9]:
# Filter on min_pct in each cell: fully empty rows are not displayed
prof.display(min_pct=5)

Make rows: 0it [00:00, ?it/s]

Make rows: 26it [00:00, 1086.51it/s]



CPU=14.7ms, CUDA=14.6ms
+------------------+--------------+---------------+--------------+---------------+-----+
| Node             |     Self CPU |           CPU |    Self CUDA |          CUDA |   # |
|------------------+--------------+---------------+--------------+---------------+-----|
| <torchprof>      |              | 14.6ms (100%) |              | 14.6ms (100%) |   1 |
|  └──AlexNet      |              |  14.2ms (97%) |              |  14.2ms (97%) |   1 |
|    ├──classifier |              |   8.0ms (55%) |              |   8.0ms (55%) |   1 |
|    │ ├──1        |  4.4ms (30%) |   4.5ms (31%) |              |   4.5ms (31%) |   1 |
|    │ ├──4        |  1.9ms (13%) |   2.0ms (14%) |              |   2.0ms (14%) |   1 |
|    ├──features   | 815.6µs (6%) |   5.8ms (39%) | 823.2µs (6%) |   5.8ms (39%) |   1 |
|    │ ├──8        |              |  850.2µs (6%) |              |  849.9µs (6%) |   1 |
|    │ ├──3        |              |  825.5µs (6%) |              |  826.4µs (6%) |  

In [11]:
# Filter items. Allow all, block nothing. These take regexes. 
prof.display(allow=[], block=[])

Make rows: 0it [00:00, ?it/s]

Make rows: 177it [00:00, 1941.75it/s]



CPU=14.7ms, CUDA=14.6ms
+-------------------------------------------+--------------+---------------------+---------------------+---------------------+-----+
| Node                                      |     Self CPU |                 CPU |           Self CUDA |                CUDA |   # |
|-------------------------------------------+--------------+---------------------+---------------------+---------------------+-----|
| <torchprof>                               | 392.2µs (3%) |       14.6ms (100%) |        393.4µs (3%) |       14.6ms (100%) |   1 |
|  ├──AlexNet                               | 236.6µs (2%) |        14.2ms (97%) |        238.5µs (2%) |        14.2ms (97%) |   1 |
|  │ ├──classifier                          | 461.0µs (3%) |         8.0ms (55%) |        462.0µs (3%) |         8.0ms (55%) |   1 |
|  │ │ ├──1                                 |  4.4ms (30%) |         4.5ms (31%) |                     |         4.5ms (31%) |   1 |
|  │ │ │ ├──aten::linear                   

In [14]:
# allow all, but block aten::
prof.display(allow=[], block=['^aten::'])

Make rows: 0it [00:00, ?it/s]

Make rows: 26it [00:00, 1245.72it/s]



CPU=14.7ms, CUDA=14.6ms
+------------------+--------------+---------------+--------------+---------------+-----+
| Node             |     Self CPU |           CPU |    Self CUDA |          CUDA |   # |
|------------------+--------------+---------------+--------------+---------------+-----|
| <torchprof>      | 392.2µs (3%) | 14.6ms (100%) | 393.4µs (3%) | 14.6ms (100%) |   1 |
|  └──AlexNet      | 236.6µs (2%) |  14.2ms (97%) | 238.5µs (2%) |  14.2ms (97%) |   1 |
|    ├──classifier | 461.0µs (3%) |   8.0ms (55%) | 462.0µs (3%) |   8.0ms (55%) |   1 |
|    │ ├──1        |  4.4ms (30%) |   4.5ms (31%) |              |   4.5ms (31%) |   1 |
|    │ ├──4        |  1.9ms (13%) |   2.0ms (14%) |              |   2.0ms (14%) |   1 |
|    │ ├──6        | 482.9µs (3%) |  564.8µs (4%) |              |  565.1µs (4%) |   1 |
|    ├──features   | 815.6µs (6%) |   5.8ms (39%) | 823.2µs (6%) |   5.8ms (39%) |   1 |
|    │ ├──8        | 679.6µs (5%) |  850.2µs (6%) |              |  849.9µs (6%) |  

In [12]:
# Sorting
prof.display(sort_by=["self_cuda_time"])

Make rows: 0it [00:00, ?it/s]

Make rows: 26it [00:00, 1041.91it/s]



CPU=14.7ms, CUDA=14.6ms
+------------------+--------------+---------------+--------------+---------------+-----+
| Node             |     Self CPU |           CPU |    Self CUDA |          CUDA |   # |
|------------------+--------------+---------------+--------------+---------------+-----|
| <torchprof>      | 392.2µs (3%) | 14.6ms (100%) | 393.4µs (3%) | 14.6ms (100%) |   1 |
|  └──AlexNet      | 236.6µs (2%) |  14.2ms (97%) | 238.5µs (2%) |  14.2ms (97%) |   1 |
|    ├──features   | 815.6µs (6%) |   5.8ms (39%) | 823.2µs (6%) |   5.8ms (39%) |   1 |
|    │ ├──9        |              |  167.6µs (1%) |              |  167.9µs (1%) |   1 |
|    │ ├──0        |              |  686.9µs (5%) |              |  687.1µs (5%) |   1 |
|    │ ├──5        |              |  237.2µs (2%) |              |  237.6µs (2%) |   1 |
|    │ ├──6        | 539.8µs (4%) |  702.5µs (5%) |              |  702.5µs (5%) |   1 |
|    │ ├──8        | 679.6µs (5%) |  850.2µs (6%) |              |  849.9µs (6%) |  

In [13]:
prof.display(color=False)

Make rows: 0it [00:00, ?it/s]

Make rows: 26it [00:00, 26966.35it/s]



CPU=14.7ms, CUDA=14.6ms
+---------------+--------------+---------------+--------------+---------------+-----+
| Node          |     Self CPU |           CPU |    Self CUDA |          CUDA |   # |
|---------------+--------------+---------------+--------------+---------------+-----|
| <torchprof>   | 392.2µs (3%) | 14.6ms (100%) | 393.4µs (3%) | 14.6ms (100%) |   1 |
| └──AlexNet    | 236.6µs (2%) |  14.2ms (97%) | 238.5µs (2%) |  14.2ms (97%) |   1 |
| ├──classifier | 461.0µs (3%) |   8.0ms (55%) | 462.0µs (3%) |   8.0ms (55%) |   1 |
| │ ├──1        |  4.4ms (30%) |   4.5ms (31%) |              |   4.5ms (31%) |   1 |
| │ ├──4        |  1.9ms (13%) |   2.0ms (14%) |              |   2.0ms (14%) |   1 |
| │ ├──6        | 482.9µs (3%) |  564.8µs (4%) |              |  565.1µs (4%) |   1 |
| ├──features   | 815.6µs (6%) |   5.8ms (39%) | 823.2µs (6%) |   5.8ms (39%) |   1 |
| │ ├──8        | 679.6µs (5%) |  850.2µs (6%) |              |  849.9µs (6%) |   1 |
| │ ├──3        | 682.8µs (5

# Running profile with other features

If we are using torch.profiler (ie. nvtx=False, by default), keyword arguments are passed through.



In [23]:
with torchprof.profile(model, use_cuda=False) as prof:
    _ = model(x)
prof.display()

Adding traces:   0%|          | 0/24 [00:00<?, ?it/s]

Adding traces: 100%|██████████| 24/24 [00:00<00:00, 63710.95it/s]


Removing traces:   0%|          | 0/24 [00:00<?, ?it/s]

Removing traces: 100%|██████████| 24/24 [00:00<00:00, 93206.76it/s]


Make Events:   0%|          | 0/252 [00:00<?, ?it/s]

...


Aggregate:   0%|          | 0/177 [00:00<?, ?it/s]

Aggregate: 100%|██████████| 177/177 [00:00<00:00, 65420.50it/s]


Make rows: 0it [00:00, ?it/s]

Make rows: 26it [00:00, 947.93it/s]



CPU=15.9ms, CUDA=0.0µs
+------------------+--------------+---------------+-------------+--------+-----+
| Node             |     Self CPU |           CPU |   Self CUDA |   CUDA |   # |
|------------------+--------------+---------------+-------------+--------+-----|
| <torchprof>      | 772.3µs (5%) | 15.8ms (100%) |             |        |   1 |
|  └──AlexNet      | 396.9µs (3%) |  15.1ms (95%) |             |        |   1 |
|    ├──features   |   1.3ms (8%) |   6.1ms (39%) |             |        |   1 |
|    │ ├──5        |              |  170.3µs (1%) |             |        |   1 |
|    │ ├──0        | 174.6µs (1%) |  485.0µs (3%) |             |        |   1 |
|    │ ├──6        | 546.3µs (3%) |  777.2µs (5%) |             |        |   1 |
|    │ ├──3        | 682.4µs (4%) |  875.5µs (6%) |             |        |   1 |
|    │ ├──2        |              |  196.8µs (1%) |             |        |   1 |
|    │ ├──10       | 487.5µs (3%) |  606.8µs (4%) |             |        |   1 |
|  

In [21]:
with torchprof.profile(model, enabled=False) as prof:
    _ = model(x)
assert prof is None

In [22]:
with torchprof.profile(model, nvtx=True) as prof:
    _ = model(x)
assert prof is None

Adding traces:   0%|          | 0/24 [00:00<?, ?it/s]

Adding traces: 100%|██████████| 24/24 [00:00<00:00, 24297.20it/s]


Removing traces:   0%|          | 0/24 [00:00<?, ?it/s]

Removing traces: 100%|██████████| 24/24 [00:00<00:00, 162098.71it/s]